In [5]:
import numpy as np
import pandas as pd


def one_hot_encoding(arr):
    s = set()
    _sd = dict()
    for e in arr:
        s.add(e)

    sl = list(s)
    for i in range(len(sl)):
        encode = np.zeros(len(sl))
        encode[i] = 1
        _sd[sl[i]] = encode

    encoded_data = []
    for k in range(len(arr)):
        encoded_data.append(_sd[arr[k]])

    return np.array(encoded_data)


def data_preprocess(filepath):
    df = pd.read_excel(filepath)
    data = df.to_numpy()

    ip = data[:, 2]

    encoded_ip = one_hot_encoding(ip)

    # print(encoded_ip)

    data[:, 2] = encoded_ip.tolist()

    # print(data)
    expanded_data = []
    for row in data:
        expanded_data.append(np.concatenate([row[:2], row[2], row[3:]]).tolist())

    expanded_data = np.array(expanded_data)

    for i in range(len(expanded_data)):
        expanded_data[i][0] = expanded_data[i][0] * 1000

    return expanded_data


filename = "outputv10"

# Data preprocessing
file_path = f"D:\\model_fit\\training\\training_data\\{filename}.xlsx"
dataset = data_preprocess(file_path)

print(dataset)

[[1.37258863e+03 5.33500000e+03 0.00000000e+00 0.00000000e+00
  1.00000000e+00 2.43902439e+00 2.58397933e+00 3.36787565e+00]
 [1.36839986e+03 2.69100000e+03 1.00000000e+00 0.00000000e+00
  0.00000000e+00 2.68948655e+00 2.40000000e+01 2.47549020e+01]
 [5.36633015e+02 3.20900000e+03 0.00000000e+00 1.00000000e+00
  0.00000000e+00 2.83505155e+00 4.83460560e+00 3.25814536e+00]
 [5.64861298e+01 8.69000000e+02 1.00000000e+00 0.00000000e+00
  0.00000000e+00 1.55038760e+00 3.71352785e+00 3.27455919e+00]
 [3.89358282e+02 2.84200000e+03 1.00000000e+00 0.00000000e+00
  0.00000000e+00 2.67639903e+00 3.09278351e+00 3.25814536e+00]
 [2.89020538e+01 5.80000000e+01 0.00000000e+00 1.00000000e+00
  0.00000000e+00 3.23383085e+00 3.54430380e+00 3.27455919e+00]
 [4.27258015e+03 9.63500000e+03 0.00000000e+00 0.00000000e+00
  1.00000000e+00 1.77664975e+00 3.53535354e+00 3.28282828e+00]
 [1.33891273e+03 5.33800000e+03 0.00000000e+00 1.00000000e+00
  0.00000000e+00 2.28426396e+00 5.01253133e+00 3.20855615e+00]


In [6]:
### Data preprocessing using MinMaxScaler
# create MinMaxScaler Object

from sklearn.preprocessing import MinMaxScaler

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()


# scaled_training_x = training_x[:, 0].reshape(-1, 1)
# scaled_training_y = training_y[:].reshape(-1, 1)

# training_x[:, 0] = scaled_training_x[:, 0]
# training_y = scaled_training_y

# Define the split points based on time
train_size = int(len(dataset) * 0.7)
val_size = int(len(dataset) * 0.2)

# Split the data
train = dataset[:train_size]
val = dataset[train_size : train_size + val_size]

# test data from other file
test = dataset[train_size + val_size :]

# x_train_first_column = x_train[:, 0].reshape(-1, 1)
# scaled_x_train_first_column = x_scaler.fit_transform(x_train_first_column)
# x_train[:, 0] = scaled_x_train_first_column.flatten()
# y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))  # reshap to 2D

# make the same sacling on valid data set and test data set
# x_val_scaled = x_scaler.transform(x_val)
# x_val_first_column = x_val[:, 0].reshape(-1, 1)
# scaled_x_val_first_column = x_scaler.fit_transform(x_val_first_column)
# x_val[:, 0] = scaled_x_val_first_column.flatten()
# y_val_scaled = y_scaler.transform(y_val.reshape(-1, 1))  # reshap to 2D


# x_test_first_column = x_test[:, 0].reshape(-1, 1)
# scaled_x_test_first_column = x_scaler.fit_transform(x_test_first_column)
# x_test[:, 0] = scaled_x_test_first_column.flatten()
# y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))  # reshap to 2D

print(train, end="\n\n")
print(val, end="\n\n")
print(test, end="\n\n")

[[1.37258863e+03 5.33500000e+03 0.00000000e+00 0.00000000e+00
  1.00000000e+00 2.43902439e+00 2.58397933e+00 3.36787565e+00]
 [1.36839986e+03 2.69100000e+03 1.00000000e+00 0.00000000e+00
  0.00000000e+00 2.68948655e+00 2.40000000e+01 2.47549020e+01]
 [5.36633015e+02 3.20900000e+03 0.00000000e+00 1.00000000e+00
  0.00000000e+00 2.83505155e+00 4.83460560e+00 3.25814536e+00]
 [5.64861298e+01 8.69000000e+02 1.00000000e+00 0.00000000e+00
  0.00000000e+00 1.55038760e+00 3.71352785e+00 3.27455919e+00]
 [3.89358282e+02 2.84200000e+03 1.00000000e+00 0.00000000e+00
  0.00000000e+00 2.67639903e+00 3.09278351e+00 3.25814536e+00]
 [2.89020538e+01 5.80000000e+01 0.00000000e+00 1.00000000e+00
  0.00000000e+00 3.23383085e+00 3.54430380e+00 3.27455919e+00]
 [4.27258015e+03 9.63500000e+03 0.00000000e+00 0.00000000e+00
  1.00000000e+00 1.77664975e+00 3.53535354e+00 3.28282828e+00]
 [1.33891273e+03 5.33800000e+03 0.00000000e+00 1.00000000e+00
  0.00000000e+00 2.28426396e+00 5.01253133e+00 3.20855615e+00]


In [7]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import (
    LSTM,
    Dense,
    Dropout,
    Input,
    Bidirectional,
    Flatten,
    TimeDistributed,
    Conv1D,
    MaxPooling1D,
    Conv2D,
    
)
from keras.models import Model
from keras.callbacks import Callback, EarlyStopping
from keras.optimizers import Adam


def create_sequences(dataset, time_steps):
    dataX, dataY = list(), list()
    for i in range(len(dataset)):
        endx = i + time_steps

        if endx > len(dataset) - 1:
            break

        dataset_x = dataset[i:endx, 1]
        dataset_y = dataset[endx, 0]

        dataX.append(dataset_x)
        dataY.append(dataset_y)

    return np.array(dataX), np.array(dataY)


# Create sequences for each set
time_steps = 1


# x_test = training_x[train_size:]
# y_test = one_hot_encoding[train_size:]

# Create sequences for each set
X_train, y_train = create_sequences(train, time_steps)
X_val, y_val = create_sequences(val, time_steps)
X_test, y_test = create_sequences(test, time_steps)

# Initialize LossHistory with validation data
class LossHistory(Callback):
    def __init__(self, x_train, y_train, x_val, y_val):
        super().__init__()
        self.train_data = (x_train, y_train)
        self.validation_data = (x_val, y_val)
        self.losses = []
        self.val_losses = []
        self.train_errors = []
        self.val_errors = []

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs["loss"])
        self.val_losses.append(logs.get("val_loss"))
        self.train_errors.append(logs.get("mean_absolute_error"))
        self.val_errors.append(logs.get("val_mean_absolute_error"))

X_train = X_train.reshape(-1, 1)
X_val = X_val.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(69, 1)
(19, 1)
(9, 1)


In [8]:
from keras.callbacks import TensorBoard
import os
import datetime


# log
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


def add_hidden_layer(
    model,
    units,
    layers,
    activation=None,
):
    if activation:
        if layers > 1:
            for layer in range(layers - 1):
                model.add(
                    LSTM(units=units, activation=activation, return_sequences=True)
                )
        model.add(LSTM(units=units, activation=activation, return_sequences=False))
    else:
        if layers > 1:
            for layer in range(layers - 1):
                model.add(LSTM(units=units, return_sequences=True))
        model.add(LSTM(units=units, return_sequences=False))


# loop params

LSTM_units = 100

Stop_patience = 20

Batch_size = 64

Learning_rate = 1e-2

Epochs = 100

Layers = 0

Activation = "relu"


# loss_function = 'mean_absolute_error'  # Assuming you are doing regression

# loss_function = 'categorical_crossentropy'

# loss_function = 'sparse_categorical_crossentropy'

loss_function = "mean_squared_error"


# Initialize LossHistory with validation data

history = LossHistory(X_train, y_train, X_val, y_val)


def model_fit():

    print("epochs:", Epochs)

    print("batch:", Batch_size)

    print("units:", LSTM_units)

    print(X_train.shape)

    # Build LSTM model
    model = Sequential()
    # Adding LSTM layer
    # model.add(LSTM(units=LSTM_units, activation=Activation, return_sequences=False))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(16, activation="linear"))
    model.add(Dense(1, activation="linear"))

    # Compile model
    model.compile(
        # optimizer=Adam(learning_rate=Learning_rate),
        optimizer="sgd",
        loss=loss_function,
        metrics=["mean_absolute_error"],
    )

    # Define early stopping

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=Stop_patience, restore_best_weights=True
    )

    # early_stopping = EarlyStopping(monitor='val_mean_absolute_error', patience=Stop_patience, restore_best_weights=True)

    # Train the model with early stopping
    print(X_train.shape)

    model.fit(
        X_train,
        y_train,
        epochs=Epochs,
        batch_size=Batch_size,
        validation_data=(X_val, y_val),
        callbacks=[history, tensorboard_callback],
        # callbacks=[history, early_stopping, tensorboard_callback],
    )

    prediction_results = model.predict(X_test)

    return prediction_results


print(X_val.shape)
prediction_results = model_fit()

(19, 1)
epochs: 100
batch: 64
units: 100
(69, 1)
(69, 1)
Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 425ms/step - loss: inf - mean_absolute_error: 207081304297056055486054400.0000 - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - loss: nan - mean_absolute_error: nan - 

In [9]:
_abs = np.abs(y_test - prediction_results)
mae = np.mean(_abs)
print("MAE:", mae)
for i in range(len(prediction_results)):
    print(f"pred: {prediction_results[i]} | real: {y_test[i]}")
avg = np.mean(y_test)
print(avg)

MAE: nan
pred: [nan] | real: 1033.835887908936
pred: [nan] | real: 697.2537040710449
pred: [nan] | real: 1049.41201210022
pred: [nan] | real: 2174.249410629272
pred: [nan] | real: 584.2568874359131
pred: [nan] | real: 3185.177803039551
pred: [nan] | real: 2835.506916046143
pred: [nan] | real: 1044.148921966553
pred: [nan] | real: 4146.541357040405
1861.1536555820041
